In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName('PartitionsAfterUnion').setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import col

data_length = 1000 

schema = StructType([StructField("value_to_process", IntegerType(), False)])
data = list(map(lambda x: [x], range(0, data_length))) # transform to list of class int

df_p10 = spark.sparkContext.parallelize(c=data, numSlices=10).toDF(schema=schema) # 10 partitions
df_p20 = spark.sparkContext.parallelize(c=data, numSlices=20).toDF(schema=schema) # 20 partitions

print(df_p10.rdd.getNumPartitions()) # equals 10
print(df_p20.rdd.getNumPartitions()) # equals 20

10
20


In [4]:
dfu = df_p10.union(df_p20)
print(dfu.rdd.getNumPartitions())

30


In [ ]:
spark.stop()